In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-prediction-challenge/sample_submission.csv
/kaggle/input/house-price-prediction-challenge/train.csv
/kaggle/input/house-price-prediction-challenge/test.csv


In [2]:
import pandas as pd 
import numpy as np 

%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score,mean_squared_error

In [3]:
df_train=pd.read_csv('../input/house-price-prediction-challenge/train.csv')

In [4]:
print(f" train data: Rows={df_train.shape[0]}, Columns={df_train.shape[1]}")

 train data: Rows=29451, Columns=12


In [5]:
df_train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO.                29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


**There is no null value neither in train nor in test dataset**

In [7]:
df_train['CITY']=df_train['ADDRESS'].apply(lambda x: x.split(',')[-1])
df_train['CITY']

0        Bangalore
1           Mysore
2        Bangalore
3        Ghaziabad
4          Kolkata
           ...    
29446         Agra
29447         Vapi
29448       Jaipur
29449      Chennai
29450       Jaipur
Name: CITY, Length: 29451, dtype: object

In [8]:
train=df_train.drop(['ADDRESS','LONGITUDE','LATITUDE'],axis=1)

In [9]:
def map_city(city):
    if city in ['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune', 'Maharashtra']:
        return 'tier1'
    elif city in ['Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Asansol', 'Aurangabad', 'Bareilly', 
                  'Belgaum', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar', 'Bikaner', 'Bilaspur', 'Bokaro Steel City', 
                  'Chandigarh', 'Coimbatore', 'Cuttack', 'Dehradun', 'Dhanbad', 'Bhilai', 'Durgapur', 'Dindigul', 'Erode', 
                  'Faridabad', 'Firozabad', 'Ghaziabad', 'Gorakhpur', 'Gulbarga', 'Guntur', 'Gwalior', 'Gurgaon', 'Guwahati', 
                  'Hamirpur', 'Hubli–Dharwad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar', 'Jamshedpur', 
                  'Jhansi', 'Jodhpur', 'Kakinada', 'Kannur', 'Kanpur', 'Karnal', 'Kochi', 'Kolhapur', 'Kollam', 'Kozhikode', 
                  'Kurnool', 'Ludhiana', 'Lucknow', 'Madurai', 'Malappuram', 'Mathura', 'Mangalore', 'Meerut', 'Moradabad', 
                  'Mysore', 'Nagpur', 'Nanded', 'Nashik', 'Nellore', 'Noida', 'Patna', 'Pondicherry', 'Purulia', 'Prayagraj', 
                  'Raipur', 'Rajkot', 'Rajahmundry', 'Ranchi', 'Rourkela', 'Ratlam', 'Salem', 'Sangli', 'Shimla', 'Siliguri', 
                  'Solapur', 'Srinagar', 'Surat', 'Thanjavur', 'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli', 'Tirunelveli', 
                  'Tiruvannamalai', 'Ujjain', 'Bijapur', 'Vadodara', 'Varanasi', 'Vasai-Virar City', 'Vijayawada', 'Visakhapatnam', 
                  'Vellore', 'Warangal']:
        return 'tier2'
    else:
        return 'tier3'

In [10]:
train['CITY']=train['CITY'].apply(map_city)

In [11]:
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,TARGET(PRICE_IN_LACS),CITY
0,Owner,0,0,2,BHK,1300.236407,1,1,55.0,tier1
1,Dealer,0,0,2,BHK,1275.000000,1,1,51.0,tier2
2,Owner,0,0,2,BHK,933.159722,1,1,43.0,tier1
3,Owner,0,1,2,BHK,929.921143,1,1,62.5,tier2
4,Dealer,1,0,2,BHK,999.009247,0,1,60.5,tier1


In [12]:
label_encoder=preprocessing.LabelEncoder()

In [13]:
train['POSTED_BY']=label_encoder.fit_transform(train['POSTED_BY'])
train['CITY']=label_encoder.fit_transform(train['CITY'])
train['BHK_OR_RK']=label_encoder.fit_transform(train['BHK_OR_RK'])
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,TARGET(PRICE_IN_LACS),CITY
0,2,0,0,2,0,1300.236407,1,1,55.0,0
1,1,0,0,2,0,1275.000000,1,1,51.0,1
2,2,0,0,2,0,933.159722,1,1,43.0,0
3,2,0,1,2,0,929.921143,1,1,62.5,1
4,1,1,0,2,0,999.009247,0,1,60.5,0


In [14]:
train['SQUARE_FT']=np.log(train['SQUARE_FT'])
train['SQUARE_FT'].head()

0    7.170301
1    7.150701
2    6.838576
3    6.835100
4    6.906764
Name: SQUARE_FT, dtype: float64

In [15]:
X=train.drop(['TARGET(PRICE_IN_LACS)'],axis=1)
y=train['TARGET(PRICE_IN_LACS)']

In [16]:
y.head()

0    55.0
1    51.0
2    43.0
3    62.5
4    60.5
Name: TARGET(PRICE_IN_LACS), dtype: float64

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(20615, 9)
(20615,)
(8836, 9)
(8836,)


In [19]:
#Linear Regression
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
y_pred

array([-470.625  ,   47.0625 ,  148.03125, ...,  -55.625  ,  -14.9375 ,
       -462.5625 ])

In [20]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the linear regression model (R-Square value):',r2*100)

Accuracy of the linear regression model (R-Square value): 52.42525127119717


In [21]:
#Random Forest Regressor 
rf_model=RandomForestRegressor(n_estimators=100,random_state=80)
rf_model.fit(X_train,y_train)
y_pred=rf_model.predict(X_test)
y_pred

array([  79.286     , 1093.55687143,   94.29      , ...,   36.18      ,
         53.798     ,   44.13671429])

In [22]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the random forest regressor model (R-Square value):',r2*100)

Accuracy of the random forest regressor model (R-Square value): 92.58180210668158


In [23]:
#Decision Tree Regressor
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
y_pred

array([  80., 1700.,  100., ...,   30.,   55.,   14.])

In [24]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the linear regression model (R-Square value):',r2*100)

Accuracy of the linear regression model (R-Square value): 88.10979811672507
